### Step
<img src='pic/nn_step.jpg' width='60%' height='60%'/>

In [15]:
# %load ../../../standard_import.txt
import numpy as np
# load MATLAB files
from scipy.io import loadmat
#%config InlineBackend.figure_formats = {'pdf',}
%matplotlib inline

data = loadmat('data/ex4data1.mat')
X = data['X']
y = data['y']%10 #这里将10还原为0
# print 'y = ',np.unique(y)
print 'X.shape = ',X.shape
print 'y.shape = ',y.shape

weights = loadmat('data/ex4weights.mat')
weights.keys()
Theta1 = weights['Theta1']
Theta2 = weights['Theta2']
print 'Theta1.shape = ',Theta1.shape
print 'Theta2.shape = ',Theta2.shape
# print np.r_[Theta1.ravel(),Theta2.ravel()].shape

X.shape =  (5000, 400)
y.shape =  (5000, 1)
Theta1.shape =  (25, 401)
Theta2.shape =  (10, 26)


## Cost function:</br>
<img src='pic/nn_cost_fun.png' width='60%' height='60%'/>

<img src='pic/sigmoid_grad.png' width='40%' height='40%'/>
<img src='pic/nn_forword.png' width='40%' height='40%'/>
后向推导结果：如果要看推导过程，可以查看：https://www.cnblogs.com/andywenzhi/p/7295262.html?utm_source=itdadao&utm_medium=referral </br>
<img src='pic/nn_backword.png' width='40%' height='40%'/>

In [6]:
def sigmoid(z):
    return(1.0 / (1 + np.exp(-z)))


def sigmoid_grad(z):
    return sigmoid(z)*(1-sigmoid(z))


def nn_cost_fun(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, reg_lambda):
    (m, n) = X.shape
    # theta1 25*401
    # theta2 10*26
    Theta1 = nn_params[0:hidden_layer_size * (input_layer_size + 1)].reshape(hidden_layer_size, input_layer_size + 1)
    Theta2 = nn_params[hidden_layer_size * (input_layer_size + 1):].reshape(num_labels, hidden_layer_size + 1)

    Y = np.zeros([m, num_labels])
    for i in range(m):
        Y[i, y[i]] = 1
        # Y[i, y[i]-1] = 1

    # 前向计算
    # a_1 5000*401
    a_1 = np.c_[np.ones(m), X]

    # z_2 5000*25
    z_2 = a_1.dot(Theta1.T)
    # a_2 5000*26
    a_2 = np.c_[np.ones(m), sigmoid(z_2)]

    # z_3 5000*10
    z_3 = a_2.dot(Theta2.T)
    a_3 = sigmoid(z_3)

    # 注意正则化项并没有计算bias项
    J = 1.0 / m * np.sum(-Y * np.log(a_3) - (1 - Y) * (np.log(1 - a_3)))
    J_reg = J + (reg_lambda / (2.0 * m)) * (np.sum(np.square(Theta1[:, 1:])) + np.sum(np.square(Theta2[:, 1:])))

    #     J = -1*(1/m)*np.sum((np.log(a3.T)*(y_matrix)+np.log(1-a3).T*(1-y_matrix))) + \
    #         (reg/(2*m))*(np.sum(np.square(theta1[:,1:])) + np.sum(np.square(theta2[:,1:])))


    # 后向计算,对z求偏导数,记为delta
    # delta_3 5000*10
    delta_3 = a_3 - Y

    # delta_2 5000*25
    delta_2 = delta_3.dot(Theta2[:, 1:]) * sigmoid_grad(z_2)

    # 后向计算,求参数的梯度,梯度的维度和thate一致
    # 首先初始化梯度
    # 10*26,25*401
    d_2 = np.zeros(Theta2.shape)
    d_1 = np.zeros(Theta1.shape)
    # Δ(2)=Δ(2)+a(2)∗δ(3) 10*26
    d_2 = d_2 + delta_3.T.dot(a_2)  # d_2初始化为0，可以不用初始化
    # Δ(1)=Δ(1)+a(1)∗δ(2) 25*401
    d_1 = d_1 + delta_2.T.dot(a_1)

    Theta1_grad = d_1 / m + 1.0*reg_lambda/m*Theta1
    Theta2_grad = d_2 / m + 1.0*reg_lambda/m*Theta2

    theta_grad = np.r_[Theta1_grad.ravel(), Theta2_grad.ravel()]
    # print 'J = ',J_reg
    return (J_reg, theta_grad)

init_params = np.r_[Theta1.ravel(),Theta2.ravel()]
nn_cost_fun(init_params, 400, 25, 10, X, y, 0)[0]


10.441459672777979

In [7]:
# ================ Part 5: Sigmoid Gradient  ================
print 'Evaluating sigmoid gradient...'
g = sigmoid_grad(np.array([-1, -0.5, 0, 0.5, 1]))
print 'Sigmoid gradient evaluated at [-1 -0.5 0 0.5 1]:'
print g

Evaluating sigmoid gradient...
Sigmoid gradient evaluated at [-1 -0.5 0 0.5 1]:
[ 0.19661193  0.23500371  0.25        0.23500371  0.19661193]


In [8]:
# ================ Part 6: Initializing Parameters Rand  ================

"""
1.One effective strategy for random initialization is to randomly select values for Θ(l) uniformly in the range [−ε, ε]. You should use ε = 0.12.
2.This range of values ensures that the parameters are kept small and makes the learning more efficient.
3.如果参数初始化为0，在bp计算过程中都一样了，这样将无意义
"""
def rand_initialize_weights(l_in, l_out):

    """
    :param l_in:
    :param l_out:
    :return:
    """
    epsilon_init = 0.12

    return np.random.rand(l_out, l_in+1) * 2 * epsilon_init -epsilon_init

rand_theta1 = rand_initialize_weights(400, 25)
rand_theta2 = rand_initialize_weights(25, 10)
rand_params = np.r_[rand_theta1.ravel(), rand_theta2.ravel()]

# print nn_cost_fun(rand_params, 400, 25, 10, X, y, 0)[0]

### ================ Part 7: compute_numerical_gradient  ================
1.使用数值方法对梯度进行检测，检测bp算法是否正确，方法就是对每一个参数值进行小的改变，然后计算值，用数学的方法求梯度<br>
2.每修改一个参数，就需要重新跑前向，后向算法来得到这个参数的梯度，所以效率非常慢，实际跑过程中尽量关闭<br>
3.If your backpropagation implementation is correct, then the relative difference will be small (less than 1e-9) Relative Difference

<img src='pic/number_grad_check.png' width='60%' height='60%'/>

In [10]:
def compute_numerical_grad(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, reg_lambda, h=0.00001):
    params_cnt = nn_params.shape[0]
    param_gradient = np.zeros(params_cnt)
    for i in range(params_cnt) :
        nn_params_temp = np.copy(nn_params)
        nn_params_temp[i] = nn_params[i] + h
        j_1 = nn_cost_fun(nn_params_temp, input_layer_size, hidden_layer_size, num_labels, X, y, reg_lambda)[0]
        nn_params_temp = np.copy(nn_params)
        nn_params_temp[i] = nn_params[i] - h
        j_2 = nn_cost_fun(nn_params_temp, input_layer_size, hidden_layer_size, num_labels, X, y, reg_lambda)[0]

        param_gradient[i] = (j_1 - j_2)/(2*h)

    return param_gradient

#测试,选择5个样本
sample = np.random.choice(X.shape[0], 5)
XX = X[sample]
yy = y[sample]
nn_param_grad = nn_cost_fun(rand_params, 400, 25, 10, XX, yy, 0)[1]
number_param_grad = compute_numerical_grad(rand_params, 400, 25, 10, XX, yy, 0)
diff = np.abs(number_param_grad-nn_param_grad)/(np.abs(number_param_grad)+np.abs(nn_param_grad))
diff

/Users/yuancailei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: RuntimeWarning: invalid value encountered in divide


array([  4.31032895e-10,              nan,              nan, ...,
         7.39688649e-11,   1.29840220e-10,   5.91227778e-11])

In [14]:
### ================ Part 8: Training Neural Network And Predict  ================
import scipy.optimize as opt

input_layer_size = 400   # 20x20 Input Images of Digits
hidden_layer_size = 25   # 25 hidden units
num_labels = 10

def predict(Theta_1, Theta_2, X):
    
    m, n = X.shape
    # 前向计算
    # a_1 5000*401
    a_1 = np.c_[np.ones(m), X]

    # z_2 5000*25
    z_2 = a_1.dot(Theta_1.T)
    # a_2 5000*26
    a_2 = np.c_[np.ones(m), sigmoid(z_2)]

    # z_3 5000*10
    z_3 = a_2.dot(Theta_2.T)
    a_3 = sigmoid(z_3)

    return np.argmax(a_3, axis=1)


l = 1
result = opt.minimize(fun=nn_cost_fun, x0=rand_params,
                      args=(input_layer_size, hidden_layer_size, num_labels, X, y, l),
                      method='TNC', jac=True, options={'maxiter': 150})
params_trained = result.x

Theta_1_trained = params_trained[0:hidden_layer_size * (input_layer_size + 1)].reshape(hidden_layer_size, input_layer_size + 1)
Theta_2_trained = params_trained[hidden_layer_size * (input_layer_size + 1):].reshape(num_labels, hidden_layer_size + 1)

# print 'Visualizing Neural Network...'
# plt.figure()
# display_data(Theta_1_trained[:, 1:], padding=1)
# plt.show()

# =================Implement Predict =================
pred = predict(Theta_1_trained, Theta_2_trained, X)
# print 'Training Set Accuracy:', np.mean(pred == y) * 100
print 'accuracy is {}%'.format(np.mean(pred == y.ravel())*100)

# print 'pred front 10 is : ', pred[3800:4050]
# print '   y front 10 is : ', y.ravel()[3800:4050]


accuracy is 98.8%
